In [1]:
#dependencies

from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
import pandas as pd
import plotly.express as px

import seaborn as sns
%matplotlib inline

import plotly.io as pio
pio.renderers.default = "notebook"


In [2]:
# Read in csv's

temp_us = "..//practice_sources/US_youtube_trending_data.csv"
temp_br = "..//practice_sources/BR_youtube_trending_data.csv"
temp_ca = "..//practice_sources/CA_youtube_trending_data.csv"
temp_de = "..//practice_sources/DE_youtube_trending_data.csv"
temp_fr = "..//practice_sources/FR_youtube_trending_data.csv"
temp_gb = "..//practice_sources/GB_youtube_trending_data.csv"
temp_in = "..//practice_sources/IN_youtube_trending_data.csv"
temp_jp = "..//practice_sources/JP_youtube_trending_data.csv"
temp_kr = "..//practice_sources/KR_youtube_trending_data.csv"
temp_mx = "..//practice_sources/MX_youtube_trending_data.csv"
temp_ru = "..//practice_sources/RU_youtube_trending_data.csv"

us_df = pd.read_csv(temp_us)
br_df = pd.read_csv(temp_br)
ca_df = pd.read_csv(temp_ca)
de_df = pd.read_csv(temp_de)
fr_df = pd.read_csv(temp_fr)
gb_df = pd.read_csv(temp_gb)
in_df = pd.read_csv(temp_in)
jp_df = pd.read_csv(temp_jp)
kr_df = pd.read_csv(temp_kr)
mx_df = pd.read_csv(temp_mx)
ru_df = pd.read_csv(temp_ru)

In [3]:
# Add country string column

us_df['country'] = 'United States'
br_df['country'] = 'Brazil'
ca_df['country'] = 'Canada'
de_df['country'] = 'Germany'
fr_df['country'] = 'France'
gb_df['country'] = 'United Kingdom'
in_df['country'] = 'India'
jp_df['country'] = 'Japan'
kr_df['country'] = 'South Korea'
mx_df['country'] = 'Mexico'
ru_df['country'] = 'Russia'

In [4]:
# Updating categoryId column with corresponding string for all csv's

category_codes = {
    "1":'Film & Animation',
    "2":'Autos & Vehicles',
    "10":'Music',
    "15":'Pets & Animals',
    "17":'Sports',
    "18":'Short Movies',
    "19":'Travel & Events',
    "20":'Gaming',
    "21":'Videoblogging',
    "22":'People & Blogs',
    "23":'Comedy',
    "24":'Entertainment',
    "25":'News & Politics',
    "26":'Howto & Style',
    "27":'Education',
    "28":'Science & Technology',
    "29":'Nonprofits & Activism',
    "30":'Movies',
    "31":'Anime/Animation',
    "32":'Action/Adventure',
    "33":'Classics',
    "34":'Comedy',
    "35":'Documentary',
    "36":'Drama',
    "37":'Family',
    "38":'Foreign',
    "39":'Horror',
    "40":'Sci-Fi/Fantasy',
    "41":'Thriller',
    "42":'Shorts',
    "43":'Shows',
    "44":'Trailers'
}

cat_code = us_df["categoryId"].map(lambda x: category_codes[str(x)])
us_df['cat_codes'] = cat_code

cat_code = br_df["categoryId"].map(lambda x: category_codes[str(x)])
br_df['cat_codes'] = cat_code

cat_code = ca_df["categoryId"].map(lambda x: category_codes[str(x)])
ca_df['cat_codes'] = cat_code

cat_code = de_df["categoryId"].map(lambda x: category_codes[str(x)])
de_df['cat_codes'] = cat_code

cat_code = fr_df["categoryId"].map(lambda x: category_codes[str(x)])
fr_df['cat_codes'] = cat_code

cat_code = gb_df["categoryId"].map(lambda x: category_codes[str(x)])
gb_df['cat_codes'] = cat_code

cat_code = in_df["categoryId"].map(lambda x: category_codes[str(x)])
in_df['cat_codes'] = cat_code

cat_code = jp_df["categoryId"].map(lambda x: category_codes[str(x)])
jp_df['cat_codes'] = cat_code

cat_code = kr_df["categoryId"].map(lambda x: category_codes[str(x)])
kr_df['cat_codes'] = cat_code

cat_code = mx_df["categoryId"].map(lambda x: category_codes[str(x)])
mx_df['cat_codes'] = cat_code

cat_code = ru_df["categoryId"].map(lambda x: category_codes[str(x)])
ru_df['cat_codes'] = cat_code

In [5]:
#concat all 11 csv's

ind_dfs = [us_df, br_df, ca_df, de_df, fr_df, gb_df, in_df, jp_df, kr_df, mx_df, ru_df]
main_df = pd.concat(ind_dfs)

In [6]:
# remove unwanted pipes from tags with .split()

def split_tags(tag):
    if tag == '[none]': return []
    return tag.lower().replace('"','').split('|')

main_df['tags'] = main_df['tags'].apply(split_tags)

In [7]:
# drop uneeded columns

main_df.drop(['comments_disabled', 'ratings_disabled'], axis = 1, inplace = True)

In [8]:
#drop uneeded columns

main_df.drop(['video_id', 'channelId'], axis = 1, inplace = True)

In [9]:
# dataframe for user behaviors (User Engagement Page)

engagement_df = main_df[['cat_codes', 'country','channelTitle', 'title', 'view_count', 'comment_count', 'likes', 'dislikes']]
engagement_df.head()

,cat_codes,country,channelTitle,title,view_count,comment_count,likes,dislikes
0,People & Blogs,United States,Brawadis,I ASKED HER TO BE MY GIRLFRIEND...,1514614,35313,156908,5855
1,Gaming,United States,Apex Legends,Apex Legends | Stories from the Outlands – “Th...,2381688,16549,146739,2794
2,Entertainment,United States,jacksepticeye,I left youtube for a month and THIS is what ha...,2038853,40221,353787,2628
3,Music,United States,XXL,XXL 2020 Freshman Class Revealed - Official An...,496771,7647,23251,1856
4,Howto & Style,United States,Mr. Kate,Ultimate DIY Home Movie Theater for The LaBran...,1123889,2196,45802,964


In [10]:
comments_views = engagement_df.groupby(by=['cat_codes'])['view_count'].count().sort_values(ascending=False)
comments_views_df = pd.DataFrame(comments_views)
comments_views_df

,view_count
cat_codes,
Entertainment,339599
Music,204069
People & Blogs,156913
Gaming,148435
Sports,129585
Comedy,78374
News & Politics,53987
Howto & Style,42393
Film & Animation,40784


In [12]:
# PAGE 2 BAR 3 comments_views

fig = px.bar(comments_views_df, x=comments_views_df.index, y = 'view_count')
fig.show()